In [1]:
%gui qt
%matplotlib notebook

import sys
sys.path.insert(0, "/home/virtual_arena/LeapSDK/lib/x64")
sys.path.insert(0, "/home/virtual_arena/LeapSDK/lib")
import Leap,pdb
import cv2, os, glob, ctypes,pickle
import LeapDrawImage as di
import numpy as np
import LeapCal
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import socket,json,array

#for notebook only
import getpass

In [2]:
def rect_corners(event,x,y,flags,param):
    global refPt, cropping
    
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt=[(x,y)]
        cropping = True #need?
        
    elif event == cv2.EVENT_LBUTTONUP:
        refPt.append((x,y))
        cropping = False
        print(refPt)
        
        cv2.rectangle(img, refPt[0],refPt[1],(0,255,0),2)
        cv2.imshow('img',img)
        
def coords_mouse_disp(event,x,y,flags,param):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        #print x,y,disp[y,x],filteredImg[y,x]
        average=0
        for u in range (-1,2):
            for v in range (-1,2):
                average += disp[y+u,x+v]
        average=average/9
        Distance= -593.97*average**(3) + 1506.8*average**(2) - 1373.1*average + 522.06
        Distance= np.around(Distance*0.01,decimals=2)
        print('Distance: '+ str(Distance)+' m')
        print('Location: ('+ str(x) +', '+ str(y)+')')
        
def undistort(image, coordinate_map, coefficient_map, width, height):
    destination = np.empty((width, height), dtype = np.ubyte)

    #wrap image data in numpy array
    i_address = int(image.data_pointer)
    ctype_array_def = ctypes.c_ubyte * image.height * image.width
    # as ctypes array
    as_ctype_array = ctype_array_def.from_address(i_address)
    # as numpy array
    as_numpy_array = np.ctypeslib.as_array(as_ctype_array)
    img = np.reshape(as_numpy_array, (image.height, image.width))

    #remap image to destination
    destination = cv2.remap(img,
                            coordinate_map,
                            coefficient_map,
                            interpolation = cv2.INTER_NEAREST)

    #resize output to desired destination size
    destination = cv2.resize(destination,
                             (width, height),
                             0, 0,
                             cv2.INTER_NEAREST)
    return destination

def getXYZ(luv,ruv):
    #calculates 3D world position given tuples of left and right (u,v) pixel positions

    luv=Leap.Vector(luv[0],luv[1],0)
    ruv=Leap.Vector(ruv[0],ruv[1],0)
    slopes_right = frame.images[1].rectify(ruv)
    slopes_left = frame.images[0].rectify(luv)

    z= 4./(slopes_right.x-slopes_left.x)
    x = z*slopes_right.x -2
    xcheck = z*slopes_left.x + 2
    ycheck = z*slopes_right.y
    y = z*slopes_left.y

#     print(y-ycheck)

    position = Leap.Vector(x,-z,y)
    
    #output in cm
    return position.x/10,y/10,position.z/10 

def unsharp(image):
#     image = img.copy()
    gaussian_3 = cv2.GaussianBlur(image, (9,9), 10.0)
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
    return unsharp_image

def getEdge(img):
    #Input img must be grayscale. Use cv2.GRAY2BGR()
    #find triangle. Modified code from anku255/shapeIdentify.py
    
    sigma = 0.33
    v = np.median(img)
    low = int(max(0, (1.0 - sigma) * v))
    high = int(min(255, (1.0 + sigma) * v))

#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    
    edged = cv2.Canny(img, low, high)
#     dilated = cv2.dilate(edged, kernel)
    (_, cnts, _) = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:1]
    return edged, cnts
    
def detectShape(c,sens=0.04):
    shape = 'unk'
    # calculate perimeter using
    peri = cv2.arcLength(c, True)
    # apply contour approximation and store the result in vertices
    vertices = cv2.approxPolyDP(c, sens * peri, True)

    # If the shape it triangle, it will have 3 vertices
    if len(vertices) == 3:
        shape = 'triangle'
    elif len(vertices)>=6:
        shape = 'circle'
#     print(len(vertices))
    return vertices

def findCont(image,cnt):
#     image = img.copy()
    vertices = []
    coord = []
    ID = 0
    print(str(len(cntR)) + ' contours found.\r')
    # loop over the contours
    for c in cnt:
        ID += 1 #label
        
        # compute the moment of contour
        M = cv2.moments(c)
        # From moment we can calculte area, centroid etc
        # The center or centroid can be calculated as follows
        if M["m00"] != 0:
            cX=(int(M['m10'] / M['m00']))
            cY=(int(M['m01'] / M['m00']))
            coord.append((cX,cY))
        else:
            coord.append((0,0))
            cX,cY = 0,0

        # call detectShape for contour c
        vertices = detectShape(c,.1) #.1 or .04
        
        #   Write the name of shape on the centoooeor of shapes
        cv2.putText(image, str(ID), (cX, cY+20), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 1)

        # Outline the contours
#         if ID == 1:
#             tri = cv2.minEnclosingTriangle(c)
#         print(c)
#         pdb.set_trace()
#         cv2.drawContours(image, tri, -1, (127, 127, 127), 1)

    #vertices not returned, but should be for planar angles for camera rotation
    return (coord,image)

In [84]:
#Live: read from Leap tracking

controller,_=di.setCtrlr()
# password = []

host = 'localhost' 
port = 50000
backlog = 5 
size = 1024 

address = ('localhost',25001)

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 
s.connect(address) 
#s.listen(backlog) 

while True:
    frame = controller.frame()
    images = frame.images
    if images[0].is_valid and images[1].is_valid:
        frameL = di.fmtImg(images[0])
        frameR = di.fmtImg(images[1])
    
    _, t_left = cv2.threshold(unsharp(frameL),180,255,cv2.THRESH_BINARY)
    _, t_right = cv2.threshold(unsharp(frameR),180,255,cv2.THRESH_BINARY)

    edgeL,cntL = getEdge(t_left)
    edgeR,cntR = getEdge(t_right)

    #vertex points not saveed for now. Use to get planar angles for camera rotations
    contL,imgL=findCont(t_left,cntL)
    contR,imgR=findCont(t_right,cntR)
    
#     if np.any(contL==0):
    
    cv2.imshow('left',imgL)
    cv2.imshow('right',imgR)    
   
    #calc 3D coordinates for first two contours only 
    XYZcoord = getXYZ(contL[0],contR[0])
    XYZcoord2 = getXYZ(contL[1],contR[1])
    old = [XYZcoord,XYZcoord2]
    
    #zeroes bug workaround
    if np.any(contL[0])==0 or np.any(contR[0])==0:
        XYZcoord = old[0]
    
    if np.any(contL[0])==0 or np.any(contR[1])==0:
        XYZcoord2 = old[1]        
        
        
#     print(XYZcoord)

    #Rotate based on LeapMotion Controller position (~40 deg, y axis rotation)
#     z' = z*cos q - x*sin q
#     x' = z*sin q + x*cos q
#     y' = y
    q = 0.698132
#     XYZcoord = list(XYZcoord)
#     XYZcoord2 = list(XYZcoord2)
#     XYZcoord[2]= XYZcoord[2]*np.cos(-q) - XYZcoord[0]*np.sin(-q)
#     XYZcoord[0]= XYZcoord[2]*np.sin(-q) + XYZcoord[0]*np.cos(-q)
#     XYZcoord2[2]= XYZcoord2[2]*np.cos(-q) - XYZcoord2[0]*np.sin(-q)
#     XYZcoord2[0]= XYZcoord2[2]*np.sin(-q) + XYZcoord2[0]*np.cos(-q)
#     print(XYZcoord)
    
    # convert to string
    XYZcoord = str("%.2f"%XYZcoord[0]) + "," + str("%.2f"%XYZcoord[1]) + "," + str("%.2f"%XYZcoord[2])
    XYZcoord2 = str("%.2f"%XYZcoord2[0]) + "," + str("%.2f"%XYZcoord2[1]) + "," + str("%.2f"%XYZcoord2[2])
#     data = bytearray()
#     data.extend(XYZcoord)
#     data.extend(',')
#     data.extend(XYZcoord2)
#     print(data)

    
    if not password:
        password = getpass.getpass()

    s.sendall(XYZcoord)
    s.sendall(XYZcoord2)


    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
        s.close()
        break

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

IndexError: list index out of range

In [81]:
s.close()

In [85]:
cv2.destroyAllWindows()

In [59]:
#Post-hoc: Read from file

os.chdir('/home/virtual_arena/Python scripts/')
left = cv2.VideoCapture('vecttest-L.mp4')
right = cv2.VideoCapture('vecttest-R.mp4')

#preallocate vector points and vertices
# vertL = np.zeros((cv2.CAP_PROP_FRAME_COUNT,1) )
# vertR = np.zeros((cv2.CAP_PROP_FRAME_COUNT,1) )
contL = []# np.zeros((cv2.CAP_PROP_FRAME_COUNT,1) )
contR = []# np.zeros((cv2.CAP_PROP_FRAME_COUNT,1) )

# Check if camera opened successfully
if (left.isOpened()== False or right.isOpened()==False): 
    print("Error opening video stream or file")

# Read until video is completed
# while True:
for i in range(int(left.get(cv2.CAP_PROP_FRAME_COUNT))):
    #grab image
    grabbedL, frameL = left.read()
    grabbedR, frameR = right.read()
    
#     frameL = cv2.cvtColor(frameL,cv2.COLOR_BGR2GRAY)   
#     frameR = cv2.cvtColor(frameR,cv2.COLOR_BGR2GRAY)       
    
    #sharpen & threshold image, then find contours and get COMs
    if grabbedL == True and grabbedR == True:
        _, t_left = cv2.threshold(unsharp(frameL),180,255,cv2.THRESH_BINARY)
        _, t_right = cv2.threshold(unsharp(frameR),180,255,cv2.THRESH_BINARY)

        edgeL,cntL = getEdge(t_left)
        edgeR,cntR = getEdge(t_right)
        
        cv2.imshow('left',edgeL)
        cv2.imshow('right',edgeR)
        
        #vertex points not saveed for now. Use to get planar angles for camera rotations
        contL.append(findCont(edgeL,cntL))
        contR.append(findCont(edgeR,cntR))
        
#         vertL[i],contL[i] = (findCont(edgeL,cntL))#getEdge(t_right))
#         vertR[i],contR[i] = (findCont(edgeL,cntL))#getEdge(t_right))
        
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        

2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

3 contours found.
3 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
3 contours found.
3 contours found.
3 contours found.
3 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours

1 contours found.
1 contours found.
2 contours found.
2 contours found.
1 contours found.
1 contours found.
1 contours found.
1 contours found.
2 contours found.
2 contours found.
1 contours found.
1 contours found.
2 contours found.
2 contours found.
1 contours found.
1 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
4 contours found.
4 contours found.
1 contours found.
1 contours found.
1 contours found.
1 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
4 contours found.
4 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours

3 contours found.
3 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
5 contours found.
5 contours found.
3 contours found.
3 contours found.
2 contours found.
2 contours found.
3 contours found.
3 contours found.
3 contours found.
3 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
5 contours found.
5 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours found.
2 contours found.
2 contours found.
4 contours found.
4 contours

KeyboardInterrupt: 

In [60]:
getXYZ(contL[0][0],contL[0][1])

0


NameError: global name 'frame' is not defined